In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported  

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 64
    RANDOM_STATE = 42


# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape
    
    def train(self):
        best_mape = float('inf')
        best_model_state = None

        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

            # Save best model state
            if val_mape < best_mape:
                best_mape = val_mape
                best_model_state = self.model.state_dict()

        # Load best model weights before prediction
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"✅ Loaded best model with Val MAPE: {best_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(1)Best.csv', index=False)
        print("✅ Submission saved to submission.csv")


# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:06<00:00,  9.21it/s]


Epoch 1/300 | Train Loss: 0.4870 | Val Loss: 0.2770 | Val MAPE: 795.46%


Epoch 2/300: 100%|██████████| 57/57 [00:05<00:00, 10.78it/s]


Epoch 2/300 | Train Loss: 0.2219 | Val Loss: 0.1584 | Val MAPE: 839.79%


Epoch 3/300: 100%|██████████| 57/57 [00:05<00:00, 10.82it/s]


...

Epoch 298/300 | Train Loss: 0.0051 | Val Loss: 0.0379 | Val MAPE: 90.39%


Epoch 299/300: 100%|██████████| 57/57 [00:05<00:00, 11.00it/s]


Epoch 299/300 | Train Loss: 0.0056 | Val Loss: 0.0356 | Val MAPE: 69.25%


Epoch 300/300: 100%|██████████| 57/57 [00:05<00:00, 10.85it/s]


Epoch 300/300 | Train Loss: 0.0061 | Val Loss: 0.0287 | Val MAPE: 78.23%
✅ Loaded best model with Val MAPE: 45.24%
✅ Submission saved to submission.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 64
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        best_mape = float('inf')
        best_model_state = None

        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

            # Save best model state
            if val_mape < best_mape:
                best_mape = val_mape
                best_model_state = self.model.state_dict()

        # Load best model weights before prediction
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"✅ Loaded best model with Val MAPE: {best_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(2)Best.csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:05<00:00, 10.32it/s]


Epoch 1/300 | Train Loss: 0.4759 | Val Loss: 0.2916 | Val MAPE: 2019.34%


Epoch 2/300: 100%|██████████| 57/57 [00:05<00:00,  9.87it/s]


Epoch 2/300 | Train Loss: 0.2177 | Val Loss: 0.1563 | Val MAPE: 669.48%


Epoch 3/300: 100%|██████████| 57/57 [00:05<00:00, 10.03it/s]


...

Epoch 298/300 | Train Loss: 0.0032 | Val Loss: 0.0367 | Val MAPE: 156.61%


Epoch 299/300: 100%|██████████| 57/57 [00:05<00:00, 11.01it/s]


Epoch 299/300 | Train Loss: 0.0031 | Val Loss: 0.0349 | Val MAPE: 122.72%


Epoch 300/300: 100%|██████████| 57/57 [00:05<00:00, 10.90it/s]


Epoch 300/300 | Train Loss: 0.0032 | Val Loss: 0.0385 | Val MAPE: 97.75%
✅ Loaded best model with Val MAPE: 45.60%
✅ Submission saved to submission.csv


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 64
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        best_mape = float('inf')
        best_model_state = None

        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

            # Save best model state
            if val_mape < best_mape:
                best_mape = val_mape
                best_model_state = self.model.state_dict()

        # Load best model weights before prediction
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"✅ Loaded best model with Val MAPE: {best_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(3)Best.csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:05<00:00, 10.43it/s]


Epoch 1/300 | Train Loss: 0.4742 | Val Loss: 0.2824 | Val MAPE: 903.00%


Epoch 2/300: 100%|██████████| 57/57 [00:05<00:00, 10.49it/s]


Epoch 2/300 | Train Loss: 0.2194 | Val Loss: 0.1546 | Val MAPE: 500.19%


Epoch 3/300: 100%|██████████| 57/57 [00:05<00:00, 10.46it/s]


...

Epoch 298/300 | Train Loss: 0.0055 | Val Loss: 0.0351 | Val MAPE: 72.44%


Epoch 299/300: 100%|██████████| 57/57 [00:05<00:00, 11.35it/s]


Epoch 299/300 | Train Loss: 0.0055 | Val Loss: 0.0294 | Val MAPE: 47.57%


Epoch 300/300: 100%|██████████| 57/57 [00:05<00:00, 11.20it/s]


Epoch 300/300 | Train Loss: 0.0053 | Val Loss: 0.0295 | Val MAPE: 142.90%
✅ Loaded best model with Val MAPE: 44.72%
✅ Submission saved to submission.csv


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 100
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        best_mape = float('inf')
        best_model_state = None

        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

            # Save best model state
            if val_mape < best_mape:
                best_mape = val_mape
                best_model_state = self.model.state_dict()

        # Load best model weights before prediction
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"✅ Loaded best model with Val MAPE: {best_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(4)Best.csv', index=False)
        print("✅ Submission saved to submission.csv")


# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:08<00:00,  6.92it/s]


Epoch 1/300 | Train Loss: 0.4995 | Val Loss: 0.2801 | Val MAPE: 2303.34%


Epoch 2/300: 100%|██████████| 57/57 [00:08<00:00,  7.08it/s]


Epoch 2/300 | Train Loss: 0.2450 | Val Loss: 0.2027 | Val MAPE: 1141.55%


Epoch 3/300: 100%|██████████| 57/57 [00:08<00:00,  7.11it/s]


...

Epoch 298/300 | Train Loss: 0.0071 | Val Loss: 0.0457 | Val MAPE: 79.20%


Epoch 299/300: 100%|██████████| 57/57 [00:08<00:00,  6.99it/s]


Epoch 299/300 | Train Loss: 0.0102 | Val Loss: 0.0504 | Val MAPE: 77.45%


Epoch 300/300: 100%|██████████| 57/57 [00:08<00:00,  7.05it/s]


Epoch 300/300 | Train Loss: 0.0090 | Val Loss: 0.0387 | Val MAPE: 127.29%
✅ Loaded best model with Val MAPE: 56.26%
✅ Submission saved to submission.csv


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 200
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        best_mape = float('inf')
        best_model_state = None

        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

            # Save best model state
            if val_mape < best_mape:
                best_mape = val_mape
                best_model_state = self.model.state_dict()

        # Load best model weights before prediction
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"✅ Loaded best model with Val MAPE: {best_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(5)Best.csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:16<00:00,  3.38it/s]


Epoch 1/300 | Train Loss: 0.5386 | Val Loss: 0.4869 | Val MAPE: 665.70%


Epoch 2/300: 100%|██████████| 57/57 [00:16<00:00,  3.39it/s]


Epoch 2/300 | Train Loss: 0.3520 | Val Loss: 0.2584 | Val MAPE: 690.89%


Epoch 3/300: 100%|██████████| 57/57 [00:17<00:00,  3.35it/s]


...

Epoch 298/300 | Train Loss: 0.0039 | Val Loss: 0.0322 | Val MAPE: 137.59%


Epoch 299/300: 100%|██████████| 57/57 [00:16<00:00,  3.48it/s]


Epoch 299/300 | Train Loss: 0.0041 | Val Loss: 0.0341 | Val MAPE: 134.87%


Epoch 300/300: 100%|██████████| 57/57 [00:16<00:00,  3.47it/s]


Epoch 300/300 | Train Loss: 0.0038 | Val Loss: 0.0345 | Val MAPE: 88.67%
✅ Loaded best model with Val MAPE: 52.70%
✅ Submission saved to submission.csv


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 200
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        best_mape = float('inf')
        best_model_state = None

        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

            # Save best model state
            if val_mape < best_mape:
                best_mape = val_mape
                best_model_state = self.model.state_dict()

        # Load best model weights before prediction
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"✅ Loaded best model with Val MAPE: {best_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(5)Best.csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:16<00:00,  3.38it/s]


Epoch 1/300 | Train Loss: 0.5376 | Val Loss: 0.4788 | Val MAPE: 593.39%


Epoch 2/300: 100%|██████████| 57/57 [00:16<00:00,  3.49it/s]


Epoch 2/300 | Train Loss: 0.3368 | Val Loss: 0.2299 | Val MAPE: 1969.04%


Epoch 3/300: 100%|██████████| 57/57 [00:15<00:00,  3.66it/s]


...

Epoch 298/300 | Train Loss: 0.0029 | Val Loss: 0.0416 | Val MAPE: 107.18%


Epoch 299/300: 100%|██████████| 57/57 [00:16<00:00,  3.44it/s]


Epoch 299/300 | Train Loss: 0.0030 | Val Loss: 0.0402 | Val MAPE: 97.58%


Epoch 300/300: 100%|██████████| 57/57 [00:16<00:00,  3.40it/s]


Epoch 300/300 | Train Loss: 0.0029 | Val Loss: 0.0371 | Val MAPE: 73.96%
✅ Loaded best model with Val MAPE: 43.25%
✅ Submission saved to submission.csv
